# Laboratorio 2
### Data science - Lynette Garcia
Daniel Gomez 21429
Abner Garcia 21285

In [56]:
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [57]:
gas = pd.read_csv('gas.csv')

# Seleccionar las columnas relevantes
columns_of_interest = ['Gasolina regular', 'Gasolina superior', 'Diesel bajo azufre', 'Gas licuado de petróleo']
data = gas[columns_of_interest]

In [58]:
data.head()

,Gasolina regular,Gasolina superior,Diesel bajo azufre,Gas licuado de petróleo
0,"177.776,50","373.963,96","0,00","194.065,74"
1,"123.115,99","243.091,07","0,00","170.703,38"
2,"161.726,42","312.084,38","0,00","161.837,37"
3,"127.338,74","285.054,89","0,00","163.048,64"
4,"168.730,19","300.913,67","0,00","171.518,86"


In [59]:
data.describe()

,Gasolina regular,Gasolina superior,Diesel bajo azufre,Gas licuado de petróleo
count,281,281,281,281
unique,281,281,78,281
top,"177.776,50","373.963,96","0,00","194.065,74"
freq,1,1,204,1


In [60]:
analysis = ProfileReport(data)
analysis.to_file('gas_analysis.html')

C:\Users\Personal\AppData\Roaming\Python\Python311\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 124.90it/s]


In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Gasolina regular         281 non-null    object
 1   Gasolina superior        281 non-null    object
 2   Diesel bajo azufre       281 non-null    object
 3   Gas licuado de petróleo  281 non-null    object
dtypes: object(4)
memory usage: 9.0+ KB
